# Enunciado

Utilizar otro dataset y
poner en práctica
la generación de
secuencias con las
estrategias presentadas.

In [ ]:
# Configuración inicial
import tensorflow as tf
import numpy as np
import os
import tensorflow.keras.layers as layers

# Cargar un dataset de texto
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print(f'Texto cargado con {len(text)} caracteres.')

# Procesamiento de texto
vocab = sorted(set(text))
char2idx = {u: i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

# Parámetros del modelo
seq_length = 100
examples_per_epoch = len(text)//seq_length
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)


1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Texto cargado con 1115394 caracteres.


# 2. Definir el Modelo LSTM para Generación de Texto
El modelo se basa en LSTM y genera texto carácter a carácter.

In [ ]:
# Construcción del modelo
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

model = tf.keras.Sequential([
    layers.Embedding(vocab_size, embedding_dim, input_length=seq_length),  # input_length se define aquí para evitar problemas
    layers.LSTM(rnn_units, return_sequences=True, recurrent_initializer='glorot_uniform'),
    layers.Dense(vocab_size)
])

# Compila el modelo nuevamente
model.compile(optimizer='adam', loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


# 3. Funciones para Estrategias de Generación
Aquí te muestro cómo implementar Greedy Search, Beam Search y Muestreo con Temperatura.

## Greedy Search

In [ ]:
def generate_text_greedy(model, start_string, num_generate=100):
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []

    for _ in range(num_generate):
        predictions = model(input_eval)
        predicted_id = tf.argmax(predictions[0, -1]).numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)


## Beam Search



In [ ]:
def generate_text_beam_search(model, start_string, num_generate=100, beam_width=3):
    sequences = [[list(start_string), 1.0]]
    for _ in range(num_generate):
        all_candidates = []
        for seq, score in sequences:
            input_eval = tf.expand_dims([char2idx[s] for s in seq], 0)
            predictions = model(input_eval)
            top_k = tf.math.top_k(predictions[0, -1], k=beam_width)
            for i in range(beam_width):
                candidate = seq + [idx2char[top_k.indices[i].numpy()]]
                candidate_score = score * top_k.values[i].numpy()
                all_candidates.append([candidate, candidate_score])
        sequences = sorted(all_candidates, key=lambda tup: tup[1], reverse=True)[:beam_width]

    return ''.join(sequences[0][0])


## Muestreo Aleatorio con Temperatura

In [ ]:
def generate_text_temperature(model, start_string, num_generate=100, temperature=1.0):
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []

    for _ in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0) / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)


## Entrenamiento del modelo

In [ ]:
for input_example, target_example in dataset.take(1):
    print("Forma de la entrada:", input_example.shape)
    print("Forma del target:", target_example.shape)

Forma de la entrada: (64, 100)
Forma del target: (64, 100)


In [ ]:
# Configuración de parámetros para el entrenamiento
BATCH_SIZE = 64
BUFFER_SIZE = 10000

# Preparación del dataset
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Configuración de los checkpoints
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")  # Añadimos .weights.h5 al final del nombre
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True)

# Crear el directorio de checkpoints si no existe
os.makedirs(checkpoint_dir, exist_ok=True)

# Entrenamiento del modelo
EPOCHS = 50
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])


Epoch 1/50


ValueError: Input 0 of layer "lstm_1" is incompatible with the layer: expected ndim=3, found ndim=4. Full shape received: (64, 64, 100, 256)

In [1]:
# Importar las librerías necesarias
import tensorflow as tf
import numpy as np
import os
import tensorflow.keras.layers as layers

# Descargar y cargar un dataset de texto
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')[:100000]  # Usar una muestra de los primeros 100,000 caracteres
print(f'Texto cargado con {len(text)} caracteres.')

# Procesamiento de texto
vocab = sorted(set(text))
char2idx = {u: i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

# Parámetros del modelo
seq_length = 100
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

# Preparación del dataset
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)
BATCH_SIZE =  64 # Reducido para acelerar el entrenamiento
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Construcción del modelo
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 512  # Reducido para un modelo más pequeño

model = tf.keras.Sequential([
    layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim),
    layers.LSTM(rnn_units, return_sequences=True, recurrent_initializer='glorot_uniform'),
    layers.Dense(vocab_size)
])

# Compilar el modelo
model.compile(optimizer='adam', loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True))

# Configura los checkpoints
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True)
os.makedirs(checkpoint_dir, exist_ok=True)

# Entrenamiento del modelo
EPOCHS = 30  # Reducido para más rapidez
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

# Funciones para generación de texto
def generate_text_greedy(model, start_string, num_generate=100):
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []

    for _ in range(num_generate):
        predictions = model(input_eval)
        predicted_id = tf.argmax(predictions[0, -1]).numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)

def generate_text_beam_search(model, start_string, num_generate=100, beam_width=3):
    sequences = [[list(start_string), 1.0]]
    for _ in range(num_generate):
        all_candidates = []
        for seq, score in sequences:
            input_eval = tf.expand_dims([char2idx[s] for s in seq], 0)
            predictions = model(input_eval)
            top_k = tf.math.top_k(predictions[0, -1], k=beam_width)
            for i in range(beam_width):
                candidate = seq + [idx2char[top_k.indices[i].numpy()]]
                candidate_score = score * top_k.values[i].numpy()
                all_candidates.append([candidate, candidate_score])
        sequences = sorted(all_candidates, key=lambda tup: tup[1], reverse=True)[:beam_width]

    return ''.join(sequences[0][0])

def generate_text_temperature(model, start_string, num_generate=100, temperature=0.5):
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []

    for _ in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0) / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)

# Generación de texto con diferentes técnicas
print("Greedy Search:")
print(generate_text_greedy(model, start_string="ROMEO: ", num_generate=200))

print("\nBeam Search:")
print(generate_text_beam_search(model, start_string="ROMEO: ", num_generate=200, beam_width=3))

print("\nMuestreo Aleatorio con Temperatura:")
print(generate_text_temperature(model, start_string="ROMEO: ", num_generate=200, temperature=0.5))


1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Texto cargado con 100000 caracteres.
Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


15/15 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - loss: 3.7762
Epoch 2/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - loss: 3.1854
Epoch 3/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - loss: 2.8271
Epoch 4/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 40s 3s/step - loss: 2.5348
Epoch 5/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - loss: 2.3869
Epoch 6/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 42s 2s/step - loss: 2.2806
Epoch 7/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - loss: 2.1928
Epoch 8/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - loss: 2.1231
Epoch 9/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - loss: 2.0567
Epoch 10/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 42s 2s/step - loss: 2.0029
Epoch 11/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - loss: 1.9622
Epoch 12/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - loss: 1.9166
Epoch 13/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - loss: 1.8766
Epoch 14/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - loss: 1.8458
Epoch 15/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - loss: 1.7943
Epoch 16/30
15/15 ━━━━━━━━━━━━

In [2]:
# Importar las librerías necesarias
import tensorflow as tf
import numpy as np
import os
import tensorflow.keras.layers as layers

# Descargar y cargar un dataset de texto
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')[:100000]  # Usar una muestra de los primeros 100,000 caracteres
print(f'Texto cargado con {len(text)} caracteres.')

# Procesamiento de texto
vocab = sorted(set(text))
char2idx = {u: i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

# Parámetros del modelo
seq_length = 100
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

# Preparación del dataset
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Construcción del modelo
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 512

model = tf.keras.Sequential([
    layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim),
    layers.LSTM(rnn_units, return_sequences=True, recurrent_initializer='glorot_uniform'),
    layers.Dense(vocab_size)
])

# Compilar el modelo
model.compile(optimizer='adam', loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True))

# Configura los checkpoints
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True)
os.makedirs(checkpoint_dir, exist_ok=True)

# Entrenamiento del modelo
EPOCHS = 30
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

# Funciones para generación de texto
def generate_text_greedy(model, start_string, num_generate=100):
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []

    for _ in range(num_generate):
        predictions = model(input_eval)
        predicted_id = tf.argmax(predictions[0, -1]).numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)

def generate_text_beam_search(model, start_string, num_generate=100, beam_width=5):  # Aumentar beam_width
    sequences = [[list(start_string), 1.0]]
    for _ in range(num_generate):
        all_candidates = []
        for seq, score in sequences:
            input_eval = tf.expand_dims([char2idx[s] for s in seq], 0)
            predictions = model(input_eval)
            top_k = tf.math.top_k(predictions[0, -1], k=beam_width)
            for i in range(beam_width):
                candidate = seq + [idx2char[top_k.indices[i].numpy()]]
                candidate_score = score * top_k.values[i].numpy()
                all_candidates.append([candidate, candidate_score])
        sequences = sorted(all_candidates, key=lambda tup: tup[1], reverse=True)[:beam_width]

    return ''.join(sequences[0][0])

def generate_text_temperature(model, start_string, num_generate=100, temperature=0.8):  # Ajustar temperatura
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []

    for _ in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0) / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)

# Generación de texto con diferentes técnicas
print("Greedy Search:")
print(generate_text_greedy(model, start_string="ROMEO: ", num_generate=200))

print("\nBeam Search:")
print(generate_text_beam_search(model, start_string="ROMEO: ", num_generate=200, beam_width=5))

print("\nMuestreo Aleatorio con Temperatura:")
print(generate_text_temperature(model, start_string="ROMEO: ", num_generate=200, temperature=0.8))


Texto cargado con 100000 caracteres.
Epoch 1/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 35s 2s/step - loss: 3.7246
Epoch 2/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 42s 2s/step - loss: 3.1185
Epoch 3/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - loss: 2.6990
Epoch 4/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - loss: 2.4505
Epoch 5/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - loss: 2.3169
Epoch 6/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - loss: 2.2270
Epoch 7/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - loss: 2.1470
Epoch 8/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - loss: 2.0785
Epoch 9/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - loss: 2.0093
Epoch 10/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 42s 2s/step - loss: 1.9591
Epoch 11/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - loss: 1.9198
Epoch 12/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - loss: 1.8764
Epoch 13/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - loss: 1.8425
Epoch 14/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - loss: 1.8053
Epoch 15/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/st

In [4]:
# Importar las librerías necesarias
import tensorflow as tf
import numpy as np
import os
import tensorflow.keras.layers as layers

# Descargar y cargar un dataset de texto
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')[:200000]  # Usar una muestra más grande de 200,000 caracteres si es posible
print(f'Texto cargado con {len(text)} caracteres.')

# Procesamiento de texto
vocab = sorted(set(text))
char2idx = {u: i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

# Parámetros del modelo
seq_length = 100
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

# Preparación del dataset
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Construcción del modelo
vocab_size = len(vocab)
embedding_dim = 512
rnn_units = 1024

model = tf.keras.Sequential([
    layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim),
    layers.LSTM(rnn_units, return_sequences=True, recurrent_initializer='glorot_uniform'),
    layers.Dense(vocab_size)
])

# Compilar el modelo
model.compile(optimizer='adam', loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True))

# Configura los checkpoints
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True)
os.makedirs(checkpoint_dir, exist_ok=True)

# Entrenamiento del modelo
EPOCHS = 50  # Aumentar el número de épocas
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

# Funciones para generación de texto
def generate_text_greedy(model, start_string, num_generate=200):
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []

    for _ in range(num_generate):
        predictions = model(input_eval)
        predicted_id = tf.argmax(predictions[0, -1]).numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)

def generate_text_beam_search(model, start_string, num_generate=200, beam_width=5):
    sequences = [[list(start_string), 1.0]]
    for _ in range(num_generate):
        all_candidates = []
        for seq, score in sequences:
            input_eval = tf.expand_dims([char2idx[s] for s in seq], 0)
            predictions = model(input_eval)
            top_k = tf.math.top_k(predictions[0, -1], k=beam_width)
            for i in range(beam_width):
                candidate = seq + [idx2char[top_k.indices[i].numpy()]]
                candidate_score = score * top_k.values[i].numpy()
                all_candidates.append([candidate, candidate_score])
        sequences = sorted(all_candidates, key=lambda tup: tup[1], reverse=True)[:beam_width]

    return ''.join(sequences[0][0])

def generate_text_temperature(model, start_string, num_generate=200, temperature=0.8):
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []

    for _ in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0) / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)

# Generación de texto con diferentes técnicas
print("Greedy Search:")
print(generate_text_greedy(model, start_string="ROMEO: ", num_generate=200))

print("\nBeam Search:")
print(generate_text_beam_search(model, start_string="ROMEO: ", num_generate=200, beam_width=5))

print("\nMuestreo Aleatorio con Temperatura:")
print(generate_text_temperature(model, start_string="ROMEO: ", num_generate=200, temperature=0.8))


Texto cargado con 200000 caracteres.
Epoch 1/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 286s 9s/step - loss: 3.5344
Epoch 2/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 278s 9s/step - loss: 2.5126
Epoch 3/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 278s 9s/step - loss: 2.2070
Epoch 4/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 321s 9s/step - loss: 2.0238
Epoch 5/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 326s 9s/step - loss: 1.8838
Epoch 6/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 321s 9s/step - loss: 1.7798
Epoch 7/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 321s 9s/step - loss: 1.6936
Epoch 8/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 320s 9s/step - loss: 1.6231
Epoch 9/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 278s 9s/step - loss: 1.5546
Epoch 10/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 319s 9s/step - loss: 1.4989
Epoch 11/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 323s 9s/step - loss: 1.4531
Epoch 12/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 277s 9s/step - loss: 1.4051
Epoch 13/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 322s 9s/step - loss: 1.3487
Epoch 14/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 277s 9s/step - loss: 1.2959
Epoch 15/50
30/30 ━━━━━━━━━━━━━━━━